## Import Classes from Libraries

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

## Read Tangled Prompt and Spec from org-mode export

In [2]:
format_instructions = open("formatter-prompt.txt", "r")
description = open("description.txt", "r")

In [3]:
model = ChatOpenAI(model="gpt-4-turbo-preview")
spec_prompt =  ChatPromptTemplate.from_messages([
  ("system", format_instructions.read()),
  ("user", "Convert the following data flow description to an interaction list: {description}")
])
parser = StrOutputParser()

ChatPromptTemplate(input_variables=['description'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a text formatting assistant that converts a plain text descriptions of a software application\'s data flow into a bulleted interaction list detailing each and every data transfer implied by the description. Each line in the output list should correspond to one leg of the data flow in the form "- <sender> sends <payload> (<format>) to <recipient>", where <sender>, <payload>, <format>, and <recipient> are placeholders for the corresponding items from the plain text description you were given. The payload <format> is optional, and if it is not specified it should be omitted from the list.  For example, if given a description that says, "The user sends a JSON query to the service, the service reads the file location from the database, and the service responds to the user with a PNG image", you should produce a bulleted list with the following

## Invoke the LLM Chat Model

In [4]:
spec_chain = spec_prompt | model | parser

In [5]:
spec =  spec_chain.invoke({"description": description.read()})

In [6]:
print(spec)

- user sends diagram description (plain text) to orchestrator service
- orchestrator service sends prompt (plain text) to formatter LLM
- formatter LLM sends list of interactions (bulleted) to orchestrator service
- orchestrator service sends interaction list to diagrammer LLM
- diagrammer LLM sends diagram source code to orchestrator service
- orchestrator service sends diagram source code to diagramming tool
- diagramming tool sends diagram image (PNG) to orchestrator service
- orchestrator service sends diagram image (PNG) to user


## Diagram Generation


In [7]:
diagram_instructions = open("diagrammer-prompt.txt", "r")
diagram_prompt = ChatPromptTemplate.from_messages([
  ("system", diagram_instructions.read()),
  ("user", "Diagram the following data flow: {spec}")
])

ChatPromptTemplate(input_variables=['spec'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a software architect\'s Data Flow Diagramming assistant that produces\ndiagram source code in the Dot language for Graphviz from a data flow\nspecification given as a bulleted list.\n\nInterpreting the Input: Each line of the input specification you receive\ndescribes an interaction which you will convert to Dot language instructions to\ndepict the data flow from one node to another. The input is in the form "-\n<sender> sends <payload> (<format>) to <recipient>", where <sender> and\n<recipient> are placeholders for nodes, and the <payload> and optional\n(<format>) describe the data that is sent between them.\n\nProducing the Output: Terminate every Dot statement with a semicolon and use the\nfollowing rules when generating the diagram.\n\nDiagram Styles:\n- the diagram\'s background should always be white\n- the diagram should always use the "R

In [8]:
diagram_chain = diagram_prompt | model | parser

ChatPromptTemplate(input_variables=['spec'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a software architect\'s Data Flow Diagramming assistant that produces\ndiagram source code in the Dot language for Graphviz from a data flow\nspecification given as a bulleted list.\n\nInterpreting the Input: Each line of the input specification you receive\ndescribes an interaction which you will convert to Dot language instructions to\ndepict the data flow from one node to another. The input is in the form "-\n<sender> sends <payload> (<format>) to <recipient>", where <sender> and\n<recipient> are placeholders for nodes, and the <payload> and optional\n(<format>) describe the data that is sent between them.\n\nProducing the Output: Terminate every Dot statement with a semicolon and use the\nfollowing rules when generating the diagram.\n\nDiagram Styles:\n- the diagram\'s background should always be white\n- the diagram should always use the "R

In [10]:
diagram_source = diagram_chain.invoke({"spec": spec})

In [11]:
print(diagram_source)

```dot
digraph DataFlow {
    graph [bgcolor=white];
    node [fontname="Roboto Mono", colorscheme=paired12, style=filled];

    user [shape=oval, color=1, label="user"];
    orchestrator_service [shape=box, color=2, label="orchestrator service"];
    formatter_LLM [shape=box, color=3, label="formatter LLM"];
    diagrammer_LLM [shape=box, color=4, label="diagrammer LLM"];
    diagramming_tool [shape=box, color=5, label="diagramming tool"];

    user -> orchestrator_service [label="plain text (diagram description)"];
    orchestrator_service -> formatter_LLM [label="plain text (prompt)"];
    formatter_LLM -> orchestrator_service [label="bulleted (list of interactions)"];
    orchestrator_service -> diagrammer_LLM [label="interaction list"];
    diagrammer_LLM -> orchestrator_service [label="diagram source code"];
    orchestrator_service -> diagramming_tool [label="diagram source code"];
    diagramming_tool -> orchestrator_service [label="PNG (diagram image)"];
    orchestrator_servi